In [1]:
import sys
sys.path.append("../src")

from news_headlines import preprocess_news, RAW_DIR
from pathlib import Path

csv_files = list(RAW_DIR.glob("*_headlines.csv"))
csv_files

outputs = [preprocess_news(path) for path in csv_files]